In [35]:
import re 
import json as js

In [18]:
# 6/6/24, 9:17 PM - +39 370 703 0730: la gpu
def get_datehour(line):
    # return date and hour object
    return line.split()[0][:-1].replace("/", "-"), line.split()[1][:-1]


In [38]:
def check_valid(line):
    # Regex for check if the line follow the format: 6/6/24, 9:17 PM - +39 370 703 0730: la gpu
    return re.match(r"\d+/\d+/\d+, \d+:\d+\s[AP]M - \+\d+ \d+ \d+: .+", line) is not None


In [23]:
file_name = "sample_whatsapp"
current_date = ""
with open(file_name + ".txt", "r", encoding="utf-8") as file:
    data = file.readlines()
    for line in data:
        # Check if the line is valid
        if not check_valid(line):
            continue
        date, hour = get_datehour(line)
        if current_date == "":
            data = date
        if current_date != date:
            past_date = date
        # Create or append to the file
        with open(file_name + "_" + past_date + ".txt", "a", encoding="utf-8") as new_file:
            new_file.write(line)
# Output



In [39]:

# Create json file

with open(file_name + "_" + past_date + ".txt", "r", encoding="utf-8") as file:
    data = file.readlines()
    json = {}
    json["dataset_name"] = "CustomDATA"
    json["dialogue_type"] = "Open-Domain"
    json["reference_summ"] = ""
    dialogue = []
    speaker = []
    for line in data:
        json_sent = {}
        #if not check_valid(line):
        #    continue
        author = line[17:].split(":")[0]
        if author not in speaker:
            speaker.append(author)
        json_sent["speaker"] = "Speaker" + str(speaker.index(author) + 1)
        print(line)
        json_sent["sentences"] = [line.split(":")[1]]
        json_sent["utterance"] = line.split(":")[1]
        json_sent["role"] = author
        dialogue.append(json_sent)
    json["dialogue"] = dialogue
    print(dialogue)
    with open(file_name + "_" + past_date + ".json", "w", encoding="utf-8") as new_file:
        js.dump(json, new_file)

[]
